In [10]:
import os
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import requests

In [5]:
load_dotenv(override=True)
api_key = os.getenv('AZURE_OPENAI_API_KEY')
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
deployment = os.getenv('AZURE_OPENAI_DEPLOYMENT')
api_version = os.getenv('AZURE_OPENAI_API_VERSION') 

In [18]:
openai = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version=api_version,
)

In [ ]:
# Prepare the chat prompt
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are an AI assistant that helps people find information."
            }
        ]
    }
]

# Include speech result if speech is enabled
messages = chat_prompt

# Generate the completion
completion = openai.chat.completions.create(
    model=deployment,
    messages=messages
)

print(completion.to_json())

In [ ]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model=deployment, messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! It’s great to hear from you! How can I assist you today?


In [11]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [12]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [13]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [16]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [20]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model=deployment, messages=messages)
print(response.choices[0].message.content)

Oh, that's a real brain teaser! The answer is 4. Not sure what you were expecting—maybe 22?


In [21]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [22]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [24]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = deployment,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [25]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\n**Overview:**\nEdward Donner is a code enthusiast and co-founder/CTO of Nebula.io, a company that leverages AI for talent discovery and management. The website showcases his interests, including coding, LLM experiments, and music production.\n\n**Key Sections:**\n- **About:** Edward describes himself and his background in coding and AI startups, mentioning his previous venture, untapt, which was acquired in 2021. He emphasizes his work with proprietary LLMs and a patented matching model.\n- **Courses and Workshops:** The website lists various courses designed to educate users on LLMs, including:\n  - **Connecting my courses – become an LLM expert and leader** (May 28, 2025)\n  - **2025 AI Executive Briefing** (May 18, 2025)\n  - **The Complete Agentic AI Engineering Course** (April 21, 2025)\n  - **LLM Workshop – Hands-on with Agents – resources** (January 23, 2025)\n\n**Connect:** Edward encourages users to follow him or connect through social 

In [26]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [27]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

**Overview:**
Edward Donner is a technology enthusiast, co-founder and CTO of Nebula.io, where he focuses on leveraging AI to aid in talent discovery and management. He has a background as the founder of AI startup untapt, which was acquired in 2021. Besides coding and LLM experimentation, he is interested in DJing and electronic music production.

**Key Highlights:**
- **Professional Background:**
  - Co-founder and CTO of Nebula.io, utilizing AI to enhance talent management.
  - Former founder and CEO of untapt, an AI startup acquired in 2021.
  - Holds patents for a proprietary matching model and has received media recognition.

- **Upcoming Events and Announcements:**
  - **May 28, 2025:** Connecting courses aimed at becoming an LLM expert and leader.
  - **May 18, 2025:** Hosting the 2025 AI Executive Briefing.
  - **April 21, 2025:** Launching "The Complete Agentic AI Engineering Course."
  - **January 23, 2025:** Conducting an LLM Workshop focused on hands-on experiences with agents.

**Interests:**
- Passionate about code, LLMs, and community engagement through social platforms. 

For further details, connect with Edward via his provided contact methods or follow him on social media.

In [28]:
display_summary("https://cnn.com")

# Summary of CNN Website

CNN is a prominent news outlet that provides comprehensive coverage of current events across various categories, including politics, health, business, science, and global affairs. The website features top stories, in-depth analyses, live updates on ongoing events, and exclusive reports.

## Recent Notable Headlines:
- **CDC Shooting Incident**: A shooter opened fire near the CDC headquarters in Atlanta, leading to a lockdown at Emory University. Reports suggest personal health concerns linked to the COVID-19 vaccine may have motivated the attack. The responding officer was fatally shot.
- **Ukraine Conflict**: Discussions regarding potential ceasefire negotiations between former President Trump and Russian President Putin in Alaska have raised concerns among Ukrainians and European allies.
- **Apollo 13 Astronaut**: The passing of famed Apollo 13 astronaut at the age of 97 was acknowledged.

## Trending Topics:
- Current global conflicts including updates on the Ukraine-Russia war and the Israel-Hamas war.
- Significant cultural discussions and lifestyle trends, including fashion and health.

The site also features multimedia content such as videos, podcasts, and interactive quizzes, enhancing user engagement and providing various ways to consume news. Additionally, there are curated sections for niche interests like travel, entertainment, and science news.